In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sqlite3

### Part 1. KARO

In [9]:
def remove_all(string):
    pattern = re.compile(r'[А-Яа-яёЁ0-9 ]+')
    return pattern.findall(string)[0].strip()

In [10]:
def find_all_theaters_KARO(theatres):
    dicti = {}
    metro_class = 'cinemalist__cinema-item__metro__station-list__station-item'
    for theater in theatres:
        data_id_list.append(theater['data-id'])
        dicti[theater.findAll('h4')[0].text] = {
            'metro': [remove_all(i.text) for i in theater.findAll('li', class_=metro_class)], 
            'address': theater.findAll('p')[0].text.split('+')[0].strip(),
            'phone': '+' + theater.findAll('p')[0].text.split('+')[-1],
            'data-id': theater['data-id']
        }
    return dicti

In [11]:
url = "https://karofilm.ru"
url_theaters = url + "/theatres"

In [12]:
data_id_list = []
r = requests.get(url_theaters)
if r.status_code == 200:
    soup = BeautifulSoup(r.text, "html.parser")
    theaters = soup.findAll('li', class_='cinemalist__cinema-item')
    karo_theatres = find_all_theaters_KARO(theaters)
else:
    print("Страница не найдена")

In [13]:
print(data_id_list)

['1', '3', '4', '6', '8', '9', '10', '11', '12', '13', '15', '33', '34', '35', '39', '44']


In [14]:
#print(theaters)
print(karo_theatres)

{' 4 на Шереметьевской': {'metro': ['Алексеевская', 'Рижская', 'Марьина роща'], 'address': 'Москва, ул. Шереметьевская, д. 20, ТЦ «Капитолий Марьина Роща», 3-й этаж.', 'phone': '+7 (909) 987-73-08', 'data-id': '1'}, ' 7 Атриум': {'metro': ['Курская', 'Чкаловская'], 'address': 'Москва, ул. Земляной вал, 33 (ТРК "АТРИУМ").', 'phone': '+7 (903) 622-23-55', 'data-id': '3'}, ' 3 Алтуфьево': {'metro': ['Алтуфьево', 'Бибирево'], 'address': 'Алтуфьевское ш., д. 70, корп. 1, ТЦ «Маркос-Молл».', 'phone': '+7 (962) 973-26-18', 'data-id': '4'}, ' 5 Иридиум': {'metro': [], 'address': 'г. Зеленоград, Крюковская пл., 1, ст. Крюково Октябрьской ж/д, ТРЦ «Иридиум», 3 этаж.', 'phone': '+7 (909) 970-70-96', 'data-id': '6'}, ' 8 Капитолий Вернадского': {'metro': ['Университет'], 'address': 'Москва, пр-т. Вернадского, д. 6, ТРК "Капитолий Вернадского".', 'phone': '+7 (909) 988-65-36', 'data-id': '8'}, ' 6 Севастопольский': {'metro': ['Крымская'], 'address': 'Москва, Севастопольский пр-т, д.11 Е, ТЦ «Капито

In [23]:
def film_time(films):
    dicti = {}
    dicti2 = {}
    for film in films:
        dicti2 = {}
        for i in film.findAll('div', class_ = 'cinema-page-item__schedule__row__board-row'):
            time_dimension = i.findAll('div', class_ = 'cinema-page-item__schedule__row__board-row__left')[0].text.strip()
            time = i.findAll('div', class_ = 'cinema-page-item__schedule__row__board-row__right')[0].findAll('a')
            time_lst = [dict(time=j.text) for j in time]
            dicti2[time_dimension] = time_lst
            dicti[film.findAll('h3')[0].text] = dicti2
    return (dicti)

In [24]:
karo_theaters_films = {}
for id_ in data_id_list:
    url_films = url + "/theatres?id=" + id_
#     print(url_films)
    r = requests.get(url_films)
    if r.status_code == 200:
        s = BeautifulSoup(r.text, "html.parser")
        films = s.findAll('div', class_='cinema-page-item__schedule__row')  
        name_cin = s.findAll('div', class_='cinema-page-item__title__left')  
        karo_films = film_time(films)
#         print(karo_films)

        karo_theaters_films[id_] = karo_films
    else:
        print("Страница не найдена")

In [25]:
# print(films)

In [18]:
for id_, films in karo_theaters_films.items():
    print(id_)
    for film, time_list in films.items():
        print(film, time_list, sep='\n', end='\n')
    print()

1
Полицейский с Рублевки. Новогодний беспредел 2, 12+
{'2D': [{'time': '10:20', 'price': None}, {'time': '12:20', 'price': None}, {'time': '14:20', 'price': None}, {'time': '16:20', 'price': None}, {'time': '18:20', 'price': None}, {'time': '20:20', 'price': None}, {'time': '22:20', 'price': None}]}
Джуманджи: Новый уровень, 12+
{'2D': [{'time': '11:25', 'price': None}, {'time': '14:00', 'price': None}, {'time': '16:35', 'price': None}, {'time': '17:40', 'price': None}, {'time': '19:10', 'price': None}, {'time': '20:15', 'price': None}, {'time': '21:45', 'price': None}]}
Холодное сердце 2 , 6+
{'2D': [{'time': '10:40', 'price': None}, {'time': '13:00', 'price': None}, {'time': '15:20', 'price': None}]}
Война токов, 12+
{'2D': [{'time': '15:40', 'price': None}]}
21 мост, 18+
{'2D': [{'time': '11:00', 'price': None}]}
Рождество на двоих, 16+
{'2D': [{'time': '20:05', 'price': None}]}
Достать ножи, 16+
{'2D': [{'time': '22:20', 'price': None}]}
Лев Яшин. Вратарь моей мечты, 6+
{'2D': [{'t

### Part 2. KINOMAX

In [68]:
def find_all_theaters_KINOMAX(theatres):
    dicti = {}

    for theater in theatres:
        data_id_list_max.append(theater.findAll('a')[0].get('href'))
        dicti[theater.findAll('a')[0].text] = {
            'metro': theater.findAll('div', class_='fs-08')[0].text.split('·')[0].strip(), 
            'address': theater.findAll('div', class_='fs-08')[-1].text.split('·')[-1].strip(),
            'data-id': theater.findAll('a')[0].get('href')
            
        }
    return dicti

In [69]:
url = "https://kinomax.ru"
url_theaters = url + "/finder"

In [70]:
data_id_list_max = []
r = requests.get(url_theaters)
if r.status_code == 200:
    soup_max = BeautifulSoup(r.text, "html.parser")
    theaters = soup_max.findAll('div', class_='pt-3 pb-3')
    kinomax_theatres = find_all_theaters_KINOMAX(theaters)
else:
    print("Страница не найдена")

In [71]:
# print(theaters)

In [72]:
print(kinomax_theatres)

{'Киномакс-XL Москва': {'metro': 'Петровско-Разумовская', 'address': 'Дмитровское шоссе, д. 89, ТРЦ «XL», 2 этаж', 'data-id': '/xl/'}, 'Киномакс-Водный Москва': {'metro': 'Водный стадион', 'address': 'ул. Головинское шоссе, д. 5, ТЦ «Водный», 3 этаж', 'data-id': '/vodny/'}, 'Киномакс-Жулебино Москва': {'metro': 'Жулебино', 'address': 'ул. Генерала Кузнецова, д. 22', 'data-id': '/zhulebino/'}, 'Киномакс-Мозаика Москва': {'metro': 'Дубровка', 'address': 'ул. 7-ая Кожуховская, д.9, ТЦ «Мозаика», 3 этаж', 'data-id': '/mozaika/'}, 'Киномакс-Пражская Москва': {'metro': 'Пражская', 'address': 'ул. Кировоградская, д.13А, ТРЦ «Columbus», 4 этаж', 'data-id': '/prazhskaya/'}, 'Киномакс-Релакс Пушкино': {'metro': 'г. Пушкино, Красноармейское шоссе, стр. 104', 'address': 'г. Пушкино, Красноармейское шоссе, стр. 104', 'data-id': '/pushkino/'}, 'Киномакс-Рига Молл Москва': {'metro': 'Строгино', 'address': 'Новорижское шоссе, 5 км от МКАД в ТК «РИГА МОЛЛ»', 'data-id': '/rigamoll/'}, 'Киномакс-Солярис 

In [289]:
def film_time_max(films):
    dicti = {}
    dicti2 = {}
    for film in films:
        dicti2 = {}
        for i in range(len(film.findAll('div', class_='d-flex w-100 schedule-row'))): #строки с dimension, time и price
            time_dimension = film.findAll('div', class_ = 'w-10 format-tag')[i].text[12:-10] # строка с dimension
            time = [j.text for j in film.findAll('div', class_='d-flex w-100 schedule-row')[i].findAll('a')] 
            #массив с всем временем для этого dimension
            price = [j.text[17:-15] for j in film.findAll('div', class_='d-flex w-100 schedule-row')[i].findAll('div', class_='fs-07 text-main pt-2 text-center')]
            #массив с всеми ценами для этого dimension

#             print(film.findAll('div', class_ = 'w-70')[0].text[1:-1], time_dimension, time, price)
            
            dicti2[time_dimension] = []
#             print(dicti2)

            if len(price) != len(time):
                time = time[1:]
            
            for j in range(len(price)):
                dicti2[time_dimension].append(dict(time=time[j], price=price[j]))
            
        dicti[film.findAll('div', class_ = 'w-70')[0].text[1:-1]] = dicti2 # название фильма
    
    print(dicti, '\n')
    return dicti

In [290]:
kinomax_theaters_films = {}
for id_ in data_id_list_max:
    url_films = url + id_
    r = requests.get(url_films)
    print(id_)
    if r.status_code == 200:
        s_max = BeautifulSoup(r.text, "html.parser")
        films_max = s_max.findAll('div', class_='d-flex border-bottom-1 border-stack film')    
        kinomax_films = film_time_max(films_max)
#         print(films_max)
#         print(kinomax_films)

        kinomax_theaters_films[id_] = kinomax_films
    else:
        print("Страница не найдена")

/xl/
{'Холодное сердце 2': {'2D': [{'time': '17:20', 'price': '250₽'}, {'time': '18:40', 'price': '300₽'}], 'VIP 2D': [{'time': '16:30', 'price': '400₽'}]}, 'Джуманджи: Новый уровень': {'3D': [{'time': '19:40', 'price': '330₽'}], '2D': [{'time': '16:00', 'price': '280₽'}], 'VIP 2D': [{'time': '20:50', 'price': '550₽'}, {'time': '23:30', 'price': '550₽'}]}, 'Полицейский с Рублевки. Новогодний Беспредел 2': {'2D': [{'time': '15:30', 'price': '250₽'}, {'time': '21:25', 'price': '300₽'}, {'time': '23:25', 'price': '300₽'}], 'VIP 2D': [{'time': '18:50', 'price': '500₽'}]}, 'Алла Пугачева. Тот самый концерт': {'2D': [{'time': '17:30', 'price': '250₽'}]}, 'Достать ножи': {'2D': [{'time': '22:20', 'price': '300₽'}]}, 'Сиротский Бруклин': {'2D': [{'time': '23:15', 'price': '300₽'}]}, 'Рождество на двоих': {'2D': [{'time': '21:00', 'price': '300₽'}]}, 'GORILLAZ: Долой фальшивых идолов (рус. субтитры)': {'2D': [{'time': '19:30', 'price': '500₽'}]}} 

/vodny/
{'Джуманджи: Новый уровень': {'3D': [{

In [288]:
for k, v in kinomax_theaters_films.items(): # вывод kinomax_theaters_films
    print(k)
    for _, __ in v.items():
        print(_, __, sep = ' ')
    print()

/xl/
Холодное сердце 2 {'2D': [{'time': '17:20', 'price': '250₽'}, {'time': '18:40', 'price': '300₽'}], 'VIP 2D': [{'time': '16:30', 'price': '400₽'}]}
Джуманджи: Новый уровень {'3D': [{'time': '19:40', 'price': '330₽'}], '2D': [{'time': '16:00', 'price': '280₽'}], 'VIP 2D': [{'time': '20:50', 'price': '550₽'}, {'time': '23:30', 'price': '550₽'}]}
Полицейский с Рублевки. Новогодний Беспредел 2 {'2D': [{'time': '15:30', 'price': '250₽'}, {'time': '21:25', 'price': '300₽'}, {'time': '23:25', 'price': '300₽'}], 'VIP 2D': [{'time': '18:50', 'price': '500₽'}]}
Алла Пугачева. Тот самый концерт {'2D': [{'time': '17:30', 'price': '250₽'}]}
Достать ножи {'2D': [{'time': '22:20', 'price': '300₽'}]}
Сиротский Бруклин {'2D': [{'time': '23:15', 'price': '300₽'}]}
Рождество на двоих {'2D': [{'time': '21:00', 'price': '300₽'}]}
GORILLAZ: Долой фальшивых идолов (рус. субтитры) {'2D': [{'time': '19:30', 'price': '500₽'}]}

/vodny/
Джуманджи: Новый уровень {'3D': [{'time': '19:30', 'price': 'от 340₽'}, 

In [168]:
con=sqlite3.connect('cinema_karo.db')
cur=conn.cursor()

In [169]:
cur.execute("""CREATE TABLE IF NOT EXISTS cinema_halls_karo( 
             cinema_id INTEGER,
             cinema_name TEXT,
             cinema_address TEXT,
             cinema_metro TEXT, 
             cinema_phone TEXT)""")

con.commit()
cur.close()
con.close()

In [154]:
a = 1
b = 'an'
c = '33'
abc = list((a, b))
cur.execute("INSERT INTO cinema_halls_karo VALUES(?, ?)", abc)

con.commit()
cur.close()
con.close()

In [167]:
cur.execute('drop table cinema_halls_karo')

In [55]:
karo_theaters_films

{'1': {'Полицейский с Рублевки. Новогодний беспредел 2, 12+': {'2D': [{'time': '12:20'},
    {'time': '14:20'},
    {'time': '16:20'},
    {'time': '18:20'},
    {'time': '20:20'},
    {'time': '22:20'}]},
  'Джуманджи: Новый уровень, 12+': {'2D': [{'time': '11:25'},
    {'time': '14:00'},
    {'time': '16:35'},
    {'time': '17:40'},
    {'time': '19:10'},
    {'time': '20:15'},
    {'time': '21:45'}]},
  'Холодное сердце 2 , 6+': {'2D': [{'time': '10:40'},
    {'time': '13:00'},
    {'time': '15:20'}]},
  'Война токов, 12+': {'2D': [{'time': '15:40'}]},
  '21 мост, 18+': {'2D': [{'time': '11:00'}]},
  'Рождество на двоих, 16+': {'2D': [{'time': '20:05'}]},
  'Достать ножи, 16+': {'2D': [{'time': '22:20'}]},
  'Лев Яшин. Вратарь моей мечты, 6+': {'2D': [{'time': '13:10'}]},
  'Ford против Ferrari, 16+': {'2D': [{'time': '22:50'}]},
  'Алла Пугачева. Тот самый концерт, 0+': {'2D': [{'time': '18:00'}]}},
 '3': {'Полицейский с Рублевки. Новогодний беспредел 2, 12+': {'2D': [{'time': '10:

In [108]:
for a, b in karo_theaters_films.items():
    print(a, b) 
    cursor.execute(f'insert or replace into cinema_halls(id) values ("{b}")')
    for c, d in enumerate(karo_theaters_films[b]):
        for e, f in enumerate(karo_theaters_films[b][d]):
            print(d)
            cursor.execute(f'insert or replace into cinema_halls(name, mode) values ("{d}", "12")')
con.commit()
cur.close()
con.close()

1 {'Полицейский с Рублевки. Новогодний беспредел 2, 12+': {'2D': [{'time': '12:20'}, {'time': '14:20'}, {'time': '16:20'}, {'time': '18:20'}, {'time': '20:20'}, {'time': '22:20'}]}, 'Джуманджи: Новый уровень, 12+': {'2D': [{'time': '11:25'}, {'time': '14:00'}, {'time': '16:35'}, {'time': '17:40'}, {'time': '19:10'}, {'time': '20:15'}, {'time': '21:45'}]}, 'Холодное сердце 2 , 6+': {'2D': [{'time': '10:40'}, {'time': '13:00'}, {'time': '15:20'}]}, 'Война токов, 12+': {'2D': [{'time': '15:40'}]}, '21 мост, 18+': {'2D': [{'time': '11:00'}]}, 'Рождество на двоих, 16+': {'2D': [{'time': '20:05'}]}, 'Достать ножи, 16+': {'2D': [{'time': '22:20'}]}, 'Лев Яшин. Вратарь моей мечты, 6+': {'2D': [{'time': '13:10'}]}, 'Ford против Ferrari, 16+': {'2D': [{'time': '22:50'}]}, 'Алла Пугачева. Тот самый концерт, 0+': {'2D': [{'time': '18:00'}]}}
3 {'Полицейский с Рублевки. Новогодний беспредел 2, 12+': {'2D': [{'time': '10:10'}, {'time': '11:55'}, {'time': '13:55'}, {'time': '14:55'}, {'time': '15:55'

In [105]:
all = cursor.execute('select * from cinema_halls').fetchall()

In [106]:
all

[(0, None, None),
 (1, None, None),
 (2, None, None),
 (3, None, None),
 (4, None, None),
 (5, None, None),
 (6, None, None),
 (7, None, None),
 (8, None, None),
 (9, None, None),
 (10, None, None),
 (11, None, None),
 (12, None, None),
 (13, None, None),
 (14, None, None),
 (15, None, None),
 (33, None, None),
 (34, None, None),
 (35, None, None),
 (39, None, None),
 (44, None, None),
 (45, 'Полицейский с Рублевки. Новогодний беспредел 2, 12+', '12'),
 (46, 'Джуманджи: Новый уровень, 12+', '12'),
 (47, 'Холодное сердце 2 , 6+', '12'),
 (48, 'Война токов, 12+', '12'),
 (49, 'Рождество на двоих, 16+', '12'),
 (50, 'Щенячий патруль: Мегащенки и отважные птенцы, 0+', '12'),
 (51, 'Достать ножи, 16+', '12'),
 (52, 'Ford против Ferrari, 16+', '12'),
 (53, 'Алла Пугачева. Тот самый концерт, 0+', '12'),
 (54, 'Полицейский с Рублевки. Новогодний беспредел 2, 12+', '12'),
 (55, 'Джуманджи: Новый уровень, 12+', '12'),
 (56, 'Холодное сердце 2 , 6+', '12'),
 (57, 'Война токов, 12+', '12'),
 (58, 